# Task 4: Optimize Portfolio Based on Forecast

## Objective
Use the TSLA return forecast from Task 3 together with historical returns for BND and SPY to construct an optimal TSLA/BND/SPY portfolio using Modern Portfolio Theory (MPT).

> Prerequisite: Run Task 1–3 first so that processed data and TSLA forecasts exist in `../data/processed/`.

In [ ]:
# 1. Imports and configuration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully for Task 4.")

Libraries imported successfully for Task 4.


In [ ]:
# 2. Load processed data and choose TSLA forecast source

# Load processed data created in Task 1
tsla_data = pd.read_csv('../data/processed/TSLA_processed.csv', index_col='Date', parse_dates=True)
bnd_data  = pd.read_csv('../data/processed/BND_processed.csv',  index_col='Date', parse_dates=True)
spy_data  = pd.read_csv('../data/processed/SPY_processed.csv',  index_col='Date', parse_dates=True)

# Ensure daily returns exist (recompute to be safe)
for df in [tsla_data, bnd_data, spy_data]:
    if 'Daily_Return' not in df.columns:
        # Use Adj Close if available, otherwise Close
        price_col = 'Adj Close' if 'Adj Close' in df.columns else 'Close'
        df['Daily_Return'] = df[price_col].pct_change()

# Historical daily returns for all three assets
returns_hist = pd.DataFrame({
    'TSLA': tsla_data['Daily_Return'],
    'BND':  bnd_data['Daily_Return'],
    'SPY':  spy_data['Daily_Return']
}).dropna()

print("Historical daily returns shape:", returns_hist.shape)

# Load model comparison results (to know which model was best). If missing, infer from cached artifacts
comparison_csv = '../data/processed/model_comparison_results.csv'
best_model_name = None
if os.path.exists(comparison_csv):
    comparison_df = pd.read_csv(comparison_csv)
    best_row = comparison_df.loc[comparison_df['RMSE'].idxmin()]
    best_model_name = best_row['Model']
else:
    print("Model comparison CSV not found; inferring best model from cached artifacts...")
    sarima_path = '../data/processed/sarima_model.pkl'
    lstm_path = '../data/processed/lstm_model.h5'
    if os.path.exists(sarima_path) and os.path.exists(lstm_path):
        print("Both SARIMA and LSTM artifacts found. Defaulting to 'SARIMA'.")
        best_model_name = 'SARIMA'
    elif os.path.exists(sarima_path):
        print("Found SARIMA model artifact. Selecting 'SARIMA'.")
        best_model_name = 'SARIMA'
    elif os.path.exists(lstm_path):
        print("Found LSTM model artifact. Selecting 'LSTM'.")
        best_model_name = 'LSTM'
    else:
        print("No model artifacts found. Defaulting to 'SARIMA'.")
        best_model_name = 'SARIMA'

print("Best model from Task 2:", best_model_name)

# Load TSLA future forecasts saved from Task 3 (if you choose to save them there later).
# For now, we will assume we regenerate them here in Task 4 (simpler workflow).

Historical daily returns shape: (2774, 3)
Model comparison CSV not found; inferring best model from cached artifacts...
Found SARIMA model artifact. Selecting 'SARIMA'.
Best model from Task 2: SARIMA


## 3. Compute Expected Returns and Covariance Matrix

- **TSLA (Forecasted Asset)**: Use the forecasted price path to derive expected daily and annual returns.
- **BND & SPY (Historical Assets)**: Use historical average daily returns, annualized.
- Compute the **annualized covariance matrix** of daily returns for all three assets, which will be used for portfolio risk.

In [ ]:
# 3.1 Compute expected annual returns
from datetime import timedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

trading_days = 252

# Reuse Adjusted Close for TSLA (fallback to Close if Adj Close not available)
if 'Adj Close' in tsla_data.columns:
    tsla_close = tsla_data['Adj Close'].dropna()
else:
    tsla_close = tsla_data['Close'].dropna()

# Recreate train/test split to stay consistent with Task 2
split_date = pd.to_datetime('2025-01-01')

# Ensure split_date matches the timezone of the index
if hasattr(tsla_close.index, 'tz') and tsla_close.index.tz is not None:
    # Index is timezone-aware, make split_date timezone-aware too
    split_date = split_date.tz_localize(tsla_close.index.tz)
# If index is timezone-naive, split_date is already naive, so no change needed

train_data = tsla_close[tsla_close.index < split_date]

forecast_steps = 252  # ~12 months horizon

future_prices = None

if best_model_name == 'SARIMA':
    print("Using SARIMA-based forecast for TSLA expected return...")

    auto_arima_model = pm.auto_arima(
        tsla_close,
        start_p=0, start_q=0,
        max_p=5, max_q=5,
        start_P=0, start_Q=0,
        max_P=2, max_Q=2,
        m=12,
        seasonal=True,
        stepwise=True,
        suppress_warnings=True,
        error_action='ignore',
        trace=False
    )

    arima_order = auto_arima_model.order
    seasonal_order = auto_arima_model.seasonal_order

    sarima_model = SARIMAX(
        tsla_close,
        order=arima_order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )

    fitted_sarima = sarima_model.fit(disp=False)
    sarima_future = fitted_sarima.get_forecast(steps=forecast_steps)
    future_prices = sarima_future.predicted_mean.values

elif best_model_name == 'LSTM':
    print("Using LSTM-based forecast for TSLA expected return...")

    scaler = MinMaxScaler(feature_range=(0, 1))
    tsla_scaled = scaler.fit_transform(tsla_close.values.reshape(-1, 1))
    train_scaled = scaler.transform(train_data.values.reshape(-1, 1))

    seq_length = 60

    def create_sequences(data, seq_len=60):
        X, y = [], []
        for i in range(seq_len, len(data)):
            X.append(data[i-seq_len:i, 0])
            y.append(data[i, 0])
        return np.array(X), np.array(y)

    X_train, y_train = create_sequences(train_scaled, seq_length)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

    lstm_model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=True),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])

    lstm_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = lstm_model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )

    last_seq = tsla_scaled[-seq_length:].reshape(1, seq_length, 1)
    future_scaled = []

    for _ in range(forecast_steps):
        next_pred = lstm_model.predict(last_seq, verbose=0)[0, 0]
        future_scaled.append(next_pred)
        last_seq = np.append(last_seq[:, 1:, :], [[[next_pred]]], axis=1)

    future_scaled = np.array(future_scaled).reshape(-1, 1)
    future_prices = scaler.inverse_transform(future_scaled).flatten()

# Convert forecast prices to daily forecast returns
future_returns_tsla = pd.Series(future_prices).pct_change().dropna()
mu_tsla_forecast_daily = future_returns_tsla.mean()
mu_tsla_forecast_annual = mu_tsla_forecast_daily * trading_days

# Historical expected returns for BND and SPY
mu_bnd_annual = returns_hist['BND'].mean() * trading_days
mu_spy_annual = returns_hist['SPY'].mean() * trading_days

print("\nApproximate expected annual returns:")
print(f"TSLA (from forecast): {mu_tsla_forecast_annual:.4f}")
print(f"BND  (historical):   {mu_bnd_annual:.4f}")
print(f"SPY  (historical):   {mu_spy_annual:.4f}")

mu_vector = np.array([mu_tsla_forecast_annual, mu_bnd_annual, mu_spy_annual])

In [ ]:
# 3.2 Annualized covariance matrix of daily returns

cov_daily = returns_hist.cov()
cov_annual = cov_daily * trading_days

print("Annualized covariance matrix:")
display(cov_annual)

## 4. Generate Efficient Frontier and Key Portfolios

We now use **PyPortfolioOpt** to:
- Build the efficient frontier from the expected return vector and annualized covariance matrix.
- Identify the **Maximum Sharpe Ratio (tangency) portfolio**.
- Identify the **Minimum Volatility portfolio**.
- Visualize the efficient frontier with these portfolios highlighted.

In [ ]:
# 4.1 Efficient Frontier, Max Sharpe, and Min Volatility Portfolios

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import plotting

assets = ['TSLA', 'BND', 'SPY']

# Build efficient frontier
ef = EfficientFrontier(mu_vector, cov_annual)

# Maximum Sharpe ratio portfolio
max_sharpe_weights = ef.max_sharpe(risk_free_rate=0.02)
cleaned_max_sharpe = ef.clean_weights()
max_sharpe_perf = ef.portfolio_performance(verbose=True)

print("\nMax Sharpe Ratio Portfolio Weights:")
for asset, w in cleaned_max_sharpe.items():
    print(f"  {asset}: {w:.4f}")

# Minimum volatility portfolio
ef_minvol = EfficientFrontier(mu_vector, cov_annual)
min_vol_weights = ef_minvol.min_volatility()
cleaned_min_vol = ef_minvol.clean_weights()
min_vol_perf = ef_minvol.portfolio_performance(verbose=True)

print("\nMinimum Volatility Portfolio Weights:")
for asset, w in cleaned_min_vol.items():
    print(f"  {asset}: {w:.4f}")

In [ ]:
# 4.2 Visualize Efficient Frontier with Key Portfolios

fig, ax = plt.subplots(figsize=(10, 6))

plotting.plot_efficient_frontier(ef_minvol, ax=ax, show_assets=False)

# Plot the two key portfolios
ax.scatter(min_vol_perf[1], min_vol_perf[0], marker='o', s=80, color='green', label='Min Volatility')
ax.scatter(max_sharpe_perf[1], max_sharpe_perf[0], marker='*', s=200, color='red', label='Max Sharpe')

ax.set_title('Efficient Frontier – TSLA/BND/SPY', fontsize=14, fontweight='bold')
ax.set_xlabel('Volatility (Risk)')
ax.set_ylabel('Expected Return')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../data/processed/task4_efficient_frontier.png', dpi=300, bbox_inches='tight')
plt.show()